# **Scraping Data Komentar Shopee**

## **Importing Library**

In [1]:
import requests
import pandas as pd
import numpy as np

### **Scrape ShopID & ProductID**

In [167]:
#Scraping
shopee_url = 'https://shopee.co.id'
keyword = 'Hijab Instant'

header = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36 Edg/101.0.1210.39', 
    'Referer' : '{}search?keyword={}'.format(shopee_url, keyword),
    'sz-token' : 'GUsGrIGz7CIeqhWCMfZ4BQ==|Bt/J+r9LI+SnjuMx1Kpc8q89/iq7/eNL4dEp920zZIG1jmRSi0t99fkwvQwq7MMFXLMrIUPUQJFPPLXwbKF7+mA=|UPCnUaD/JM6rovp2|05|3'
}
#URL Api Shopee
urlApiSeacrh = 'https://shopee.co.id/api/v4/search/search_items?by=relevancy&keyword={}&limit=100&newest=0&order=desc&page_type=search&scenario=PAGE_GLOBAL_SEARCH&version=2'.format(keyword)
urlApiRatings = 'https://shopee.co.id/api/v2/item/get_ratings'

In [173]:
# Fungsi Scrape data shopid & productid
def Scrape_Id():
  # API request 
  r = requests.get(urlApiSeacrh, headers = header).json()
  shopid, productid  = [], []

  for item in r['items']:
      shopid.append(item['item_basic']['shopid'])
      productid.append(item['item_basic']['itemid'])
      
  framex = pd.DataFrame(shopid, columns=['shop_id'])
  framex['product_id'] = productid
  return framex

In [4]:
# framex

### **Scrape Rating**

In [180]:
def Scrape_Rating(url,dataID):
  framey = pd.DataFrame(columns=['Akun','Isi Ulasan','Tag Ulasan','Bintang'])
  
  for x in range(0,len(dataID)):
    # Parameter
    params = {
      "filter": "0",
      "flag": "1",
      "itemid": dataID['product_id'][x],
      "limit": "20", #->limit jumlah ulasan per product 
      "offset": "10",
      "shopid": dataID['shop_id'][x],
      "type": "0"
      }
    # API request
    r = requests.get(url, params=params).json()
    frame = pd.DataFrame(columns=['Akun','Isi Ulasan','Tag Ulasan','Bintang'])
      
    for item in r['data']['ratings']:
      tags = ''
      tagx = item['tags']
      if tagx != None:
        for x in range(0, len(tagx)):
          tag = tagx[x].get('tag_description')
          tags += tag + ' '
      else:
        tags = np.nan
      frame.loc[len(frame.index)] = [item['author_username'], item['comment'], tags, item['rating_star']]
    framey = framey.append(frame,ignore_index=True)
  return framey

In [ ]:
# DataID

## **Pemanggilan Fungsi**

In [181]:
# DataID = Scrape_Id()
Data = Scrape_Rating(urlApiRatings,DataID)

In [184]:
Data

,Akun,Isi Ulasan,Tag Ulasan,Bintang
0,l*****h,Alhamdulillah barangnya sampai..maaf ga sesuai...,Kualitas produk sangat baik. Produk original.,5
1,j*****c,"Bahannya lumayan bagus, kepalanya gede banget,...",Kualitas produk sangat baik. Produk original. ...,5
2,qintaaf,"Alhamdulillah paketnya cepet sampai, harga lum...",Kualitas produk sangat baik. Produk original. ...,5
3,wec9rphkbv,Bagus bngt kerudung nya 😍 sama seperti yang ak...,Harga produk sangat baik. Kecepatan pengiriman...,5
4,f*****a,Bahannya adem\nNyaman di pake\nCocok untuk sko...,Kualitas produk sangat baik. Harga produk sang...,5
...,...,...,...,...
1939,efrylala,,NaN,5
1940,novita_uli_sihombing,Sesuai brg nya\nThanks toko,NaN,5
1941,ferinariani,"Produk baik, harga baik, pengemasan rapi, peng...",NaN,5
1942,c*****y,Alhamduah paket mendarat dengan selamat. Terim...,NaN,5


## **Eksport dataframe kedalan CSV**

In [183]:
Data.to_csv('data_ulasan.csv')